In [1]:
# import modules
import copy
import time
from sys import path
from collections import defaultdict

In [2]:
import pandas as pd
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [3]:
path.append('../code/src')

In [4]:
import svd_constraint
from inter_rec import InterRec
from evaluate import Evaluation as ev

In [5]:
# constants
# file path
input_path = "../data/"
output_path = '../result/'
rate_file = 'reduced_rating_data.csv'
attr_file = 'recipe_data.csv'

# user/item max
user_max = 1000
food_max = 5000

# const count
const_count = 3

In [6]:
# Dictionary keys
rs1 = 'CnstSVD'
rs2 = 'CnstSVD-all'
rs3 = 'CnstSVD-weaker'
rs4 = 'CnstSVD-weak'
val_r1 = 'RMSE'
val_r2 = 'nDCG'
val_t1 = 'train time(s)'
val_t2 = 'exec time(s)'

In [7]:
val_dict = {
    val_r1: 0,
    val_r2: 0,
    val_t1: 0,
    val_t2: 0
}

keys = [rs1]
keys = keys + [rs2]
keys = keys + [rs3]
keys = keys + [rs4]

result = {key: copy.deepcopy(val_dict) for key in keys}

In [8]:
# read previous result
result_df = pd.read_csv(output_path + 'SVD_evaluation.csv', index_col = 0).transpose()
result.update(result_df.to_dict())
result

{'CnstSVD': {'RMSE': 0.9205901113054168,
  'nDCG': 0.0069285946967008,
  'train time(s)': 101.932088692983,
  'exec time(s)': 4.953307310740153},
 'CnstSVD-all': {'RMSE': 0.9206537684416954,
  'nDCG': 0.0060981969923417,
  'train time(s)': 104.46123909950256,
  'exec time(s)': 5.509816884994507},
 'CnstSVD-weaker': {'RMSE': 0.9207082289951668,
  'nDCG': 0.006522022532252,
  'train time(s)': 102.43809390068054,
  'exec time(s)': 4.956319252649943},
 'CnstSVD-weak': {'RMSE': 0.9224348065424328,
  'nDCG': 0.0045742313609021,
  'train time(s)': 101.47139080365498,
  'exec time(s)': 5.133922338485718}}

# Evaluate CnstSVD

In [9]:
def run_inter(idx, algo):    
    # InterRec applies constraint after the rating of each item is predicted
    rec = InterRec(input_path + rate_file, input_path + attr_file, 
                           input_path + 'const' + '.' + str(idx) + '.csv', 
                            algo, split = True)
    
    rec.get_data()  # get rating, attribute, recipe data
    
    # train with data
    start = time.time()
    rec.train()  
    t1 = time.time() - start
    
    # predict rating for test-set
    predict_test = rec.test_rmse()
    r1 = ev.calculate_rmse(predict_test)
    
    # get top-n for anti-test-set
    start = time.time()
    rec.test()
    top_n_df = rec.get_top_n()
    t2 = time.time() - start

    # calculate nDCG
    r2 = ev.calculate_ndcg(rec.get_rel(), top_n_df, 10)
        
    return r1, r2, t1, t2

In [10]:
r1_sum = 0
r2_sum = 0
t1_sum = 0
t2_sum = 0
for j in range (1, const_count + 1):
    r1, r2, t1, t2 = run_inter(j, svd_constraint.CnstSVD())
    r1_sum = r1_sum + r1
    r2_sum = r2_sum + r2
    t1_sum = t1_sum + t1
    t2_sum = t2_sum + t2
    print('Const'+'.'+str(j)+" done")

result[rs1][val_r1] = r1_sum/const_count
result[rs1][val_r2] = r2_sum/const_count
result[rs1][val_t1] = t1_sum/const_count
result[rs1][val_t2] = t2_sum/const_count

Const.1 done
Const.2 done
Const.3 done


# Evaluate CnstSVD_all

In [11]:
r1_sum = 0
r2_sum = 0
t1_sum = 0
t2_sum = 0
for j in range (1, const_count + 1):
    r1, r2, t1, t2 = run_inter(j, svd_constraint.CnstSVD_all())
    r1_sum = r1_sum + r1
    r2_sum = r2_sum + r2
    t1_sum = t1_sum + t1
    t2_sum = t2_sum + t2
    print('Const'+'.'+str(j)+" done")

result[rs2][val_r1] = r1_sum/const_count
result[rs2][val_r2] = r2_sum/const_count
result[rs2][val_t1] = t1_sum/const_count
result[rs2][val_t2] = t2_sum/const_count

Const.1 done
Const.2 done
Const.3 done


# Evaluate CnstSVD_weaker

In [12]:
r1_sum = 0
r2_sum = 0
t1_sum = 0
t2_sum = 0
for j in range (1, const_count + 1):
    r1, r2, t1, t2 = run_inter(j, svd_constraint.CnstSVD_weaker())
    r1_sum = r1_sum + r1
    r2_sum = r2_sum + r2
    t1_sum = t1_sum + t1
    t2_sum = t2_sum + t2
    print('Const'+'.'+str(j)+" done")

result[rs3][val_r1] = r1_sum/const_count
result[rs3][val_r2] = r2_sum/const_count
result[rs3][val_t1] = t1_sum/const_count
result[rs3][val_t2] = t2_sum/const_count

Const.1 done
Const.2 done
Const.3 done


# Evaluate CnstSVD_weak

In [14]:
r1_sum = 0
r2_sum = 0
t1_sum = 0
t2_sum = 0
for j in range (1, const_count + 1):
    r1, r2, t1, t2 = run_inter(j, svd_constraint.CnstSVD_weak())
    r1_sum = r1_sum + r1
    r2_sum = r2_sum + r2
    t1_sum = t1_sum + t1
    t2_sum = t2_sum + t2
    print('Const'+'.'+str(j)+" done")

result[rs4][val_r1] = r1_sum/const_count
result[rs4][val_r2] = r2_sum/const_count
result[rs4][val_t1] = t1_sum/const_count
result[rs4][val_t2] = t2_sum/const_count

Const.1 done
Const.2 done
Const.3 done


# Result

In [15]:
result_df = pd.DataFrame.from_dict(result, orient='index', columns=['RMSE', 'nDCG', 'train time(s)', 'exec time(s)'])
result_df.to_csv(output_path + 'SVD_evaluation.csv')
result_df

,RMSE,nDCG,train time(s),exec time(s)
CnstSVD,2.755802,0.029961,312.547766,14.778024
CnstSVD-all,2.755727,0.025348,309.044494,14.776944
CnstSVD-weaker,2.755852,0.025907,298.256033,14.102205
CnstSVD-weak,2.761731,0.008917,289.604155,13.581923
